In [1]:
from pprint import pprint
import lerobot

# List all available datasets
print("List of available datasets:")
pprint(lerobot.available_datasets)

List of available datasets:
['lerobot/aloha_mobile_cabinet',
 'lerobot/aloha_mobile_chair',
 'lerobot/aloha_mobile_elevator',
 'lerobot/aloha_mobile_shrimp',
 'lerobot/aloha_mobile_wash_pan',
 'lerobot/aloha_mobile_wipe_wine',
 'lerobot/aloha_sim_insertion_human',
 'lerobot/aloha_sim_insertion_human_image',
 'lerobot/aloha_sim_insertion_scripted',
 'lerobot/aloha_sim_insertion_scripted_image',
 'lerobot/aloha_sim_transfer_cube_human',
 'lerobot/aloha_sim_transfer_cube_human_image',
 'lerobot/aloha_sim_transfer_cube_scripted',
 'lerobot/aloha_sim_transfer_cube_scripted_image',
 'lerobot/aloha_static_battery',
 'lerobot/aloha_static_candy',
 'lerobot/aloha_static_coffee',
 'lerobot/aloha_static_coffee_new',
 'lerobot/aloha_static_cups_open',
 'lerobot/aloha_static_fork_pick_up',
 'lerobot/aloha_static_pingpong_test',
 'lerobot/aloha_static_pro_pencil',
 'lerobot/aloha_static_screw_driver',
 'lerobot/aloha_static_tape',
 'lerobot/aloha_static_thread_velcro',
 'lerobot/aloha_static_towel',

In [2]:
from lerobot.common.datasets.lerobot_dataset import LeRobotDatasetMetadata

repo_id = "lerobot/aloha_sim_transfer_cube_human_image"
ds_meta = LeRobotDatasetMetadata(repo_id)

# Display metadata information
print(f"Total number of episodes: {ds_meta.total_episodes}")
print(f"Average number of frames per episode: {ds_meta.total_frames / ds_meta.total_episodes:.3f}")
print(f"Frames per second: {ds_meta.fps}")
print(f"Robot type: {ds_meta.robot_type}")
print("Tasks:")
print(ds_meta.tasks)
print("Features:")
pprint(ds_meta.features)


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Total number of episodes: 50
Average number of frames per episode: 400.000
Frames per second: 50
Robot type: aloha
Tasks:
{0: 'Pick up the cube with the right arm and transfer it to the left arm.'}
Features:
{'action': {'dtype': 'float32',
            'names': {'motors': ['left_waist',
                                 'left_shoulder',
                                 'left_elbow',
                                 'left_forearm_roll',
                                 'left_wrist_angle',
                                 'left_wrist_rotate',
                                 'left_gripper',
                                 'right_waist',
                                 'right_shoulder',
                                 'right_elbow',
                                 'right_forearm_roll',
                                 'right_wrist_angle',
                                 'right_wrist_rotate',
                                 'right_gripper']},
            'shape': (14,)},
 'episode_inde

In [3]:
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset

# Load a subset of episodes
dataset = LeRobotDataset(repo_id, episodes=[0, 10, 11, 23])
print(f"Selected episodes: {dataset.episodes}")
print(f"Number of frames selected: {dataset.num_frames}")

# Load the entire dataset
dataset = LeRobotDataset(repo_id)
print(f"Number of frames: {dataset.num_frames}")

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Selected episodes: [0, 10, 11, 23]
Number of frames selected: 1600


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Fetching 56 files:   0%|          | 0/56 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

Number of frames: 20000


Accessing Data

In [4]:
# Access frames by episode number
episode_index = 0
from_idx = dataset.episode_data_index["from"][episode_index].item()
to_idx = dataset.episode_data_index["to"][episode_index].item()
print(f"Episode {episode_index}: {from_idx} to {to_idx}")

# Access image frames from the first camera
print(f'camera_keys: {dataset.meta.camera_keys}')
camera_key = dataset.meta.camera_keys[0]
frames = [dataset[idx][camera_key] for idx in range(from_idx, to_idx)]

# Print frame type and shape
print(type(frames[0]))
print(frames[0].shape)


Episode 0: 0 to 400
camera_keys: ['observation.images.top']
<class 'torch.Tensor'>
torch.Size([3, 480, 640])


In [5]:
pprint(dataset[200].keys())
print(dataset[200]['observation.state'].shape)
print(dataset[200]['action'].shape)
print(dataset[200]['timestamp'])
print(dataset[200]['frame_index'])

dict_keys(['observation.images.top', 'observation.state', 'action', 'episode_index', 'frame_index', 'timestamp', 'next.done', 'index', 'task_index'])
torch.Size([14])
torch.Size([14])
tensor(4.)
tensor(200)


Using Timestamp-Based Frame Selection, not recommended in ACT paper

In [12]:
delta_timestamps = {
    # loads 4 images: 1 second before current frame, 500 ms before, 200 ms before, and current frame
    camera_key: [-1, -0.5, -0.30, 0],
    "observation.state": [-1.5, -1, -0.5, -0.20, -0.10, 0],
    "action": [t / dataset.fps for t in range(64)],
}

dataset = LeRobotDataset(repo_id, delta_timestamps=delta_timestamps)
print(f"{dataset[0][camera_key].shape=}")  # (4, c, h, w)
print(f"{dataset[0]['observation.state'].shape=}")  # (6, c)
print(f"{dataset[0]['action'].shape=}\n")  # (64, c)


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Fetching 56 files:   0%|          | 0/56 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

dataset[0][camera_key].shape=torch.Size([4, 3, 480, 640])
dataset[0]['observation.state'].shape=torch.Size([6, 14])
dataset[0]['action'].shape=torch.Size([64, 14])



In [6]:
import torch

dataloader = torch.utils.data.DataLoader(
    dataset,
    num_workers=1,
    batch_size=1,
    shuffle=True,
)

for batch in dataloader:
    print(f"{batch[camera_key].shape=}")  # (1, 4, c, h, w)
    print(f"{batch['observation.state'].shape=}")  # (1, 5, c)
    print(f"{batch['action'].shape=}")  # (1, 64, c)
    break


batch[camera_key].shape=torch.Size([1, 3, 480, 640])
batch['observation.state'].shape=torch.Size([1, 14])
batch['action'].shape=torch.Size([1, 14])


In [7]:
print(len(dataloader))

20000
